In [7]:
import json

import torch
import wandb

from src.data_processing.data_loader import data_loader
from src.models.Dir_VRNN.model import DirVRNN


In [40]:
# Load Configuration 
with open("src/models/DIR_VRNN/run_config.json", "r") as f:
    run_config = json.load(f)
    f.close()

# Extract info
data_config = run_config["data_config"]
model_config = run_config["model_config"]
training_config = run_config["training_config"]

In [33]:

# Load and process data
data_info = data_loader(**data_config)

100%|██████████| 8328/8328 [00:30<00:00, 269.96it/s]



MIMIC data successfully loaded.

Basic information 
 Input shape: (8328, 10, 9), (8328, 4) 
 Outcome Distribution: Death          26
ICU          1321
Ward         4121
Discharge    2860
dtype: int64


In [38]:
# GPU and model setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(run_config["model_config"]["seed"])
torch.backends.cudnn.deterministic = True


In [43]:
from importlib import reload
import src.models.Dir_VRNN.model as BaseModels

reload(BaseModels)


model = BaseModels.DirVRNN(**model_config).to(device)
model.fit(data_info=data_info, train_info=training_config, run_config=run_config)
torch.save(model.state_dict(), "DirVRNN.h5")

AttributeError: module 'src.models.Dir_VRNN' has no attribute 'DirVRNN'

In [36]:
training_config

{'lr': 0.001, 'epochs': 5, 'bs': 128}